In [1]:
import os
os.environ['OPENAI_API_KEY'] = ""
os.environ['ACTIVELOOP_TOKEN'] = ""

In [2]:
from langchain import OpenAI, LLMChain
from langchain.chains.mapreduce import MapReduceChain
from langchain.prompts import PromptTemplate
from langchain.chains.summarize import load_summarize_chain

llm = OpenAI(model_name="gpt-3.5-turbo-instruct", temperature=0)

/home/gustavo-barbosa/anaconda3/envs/genai/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [3]:
import yt_dlp

def download_mp4_from_youtube(urls, job_id):
    # This will hold the titles and authors of each downloaded video
    video_info = []

    for i, url in enumerate(urls):
        # Set the options for the download
        file_temp = f'./{job_id}_{i}.mp4'
        ydl_opts = {
            'format': 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/best[ext=mp4]',
            'outtmpl': file_temp,
            'quiet': True,
        }

        # Download the video file
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            result = ydl.extract_info(url, download=True)
            title = result.get('title', "")
            author = result.get('uploader', "")

        # Add the title and author to our list
        video_info.append((file_temp, title, author))

    return video_info

urls=["https://www.youtube.com/watch?v=Mi1R5Wt8PEI",
    "https://www.youtube.com/watch?v=o0LjwAohJO8",]
vides_details = download_mp4_from_youtube(urls, 1)

In [4]:
import whisper

# load the model
model = whisper.load_model("base")

# iterate through each video and transcribe
results = []
for video in vides_details:
    result = model.transcribe(video[0])
    results.append( result['text'] )
    print(f"Transcription for {video[0]}:\n{result['text']}\n")

Transcription for ./1_0.mp4:
 O ano 1983, o governo americano do presidente Ronald Reagan em comedou a produção de um jogo de taboleiro chamado Proud Profit, ou Profeta Urgulhosa em Matarovção Livre. Essa não é uma curiosidade histórica engraçada, mas uma das histórias mais aterruizadoras que saíram da Guerra Free. No mesmo ano, existiam mais de 60 mil armas no cléalis entre os Estados Unidos e a União Soviética. E o jogo Proud Profit, criado pelo doutor de economia por Harvard, Thomas Shelling, tinha como objetivo simular uma guerra nuclear no contexto da Guerra Free. Durante duas semanas, mais de 200 pessoas se reuniram no colégio nacional de guerra em Washington para jogar Proud Profit, partindo de diferentes cenários para o começo de uma guerra nuclear. O cenário de jogados nunca foram revelados pela governo americano. Uma das poucas coisas que nós sabemos da simulação de guerra nuclear vindo os relatos de Paul Breaking, um professor de ciências políticas de erro. Segundo Breaking,

In [5]:
with open('text.txt', 'w') as file:
    for transcription in results:
        file.write(transcription + '\n')

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=0, separators=[" ", ",", "\n"]
    )

# Load the texts
with open('text.txt') as f:
    text = f.read()
texts = text_splitter.split_text(text)

In [7]:
from langchain.docstore.document import Document

docs = [Document(page_content=t) for t in texts[:4]]

In [8]:
from langchain.vectorstores import DeepLake
from langchain.embeddings.openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model='text-embedding-ada-002')

# create Deep Lake dataset
# TODO: use your organization id here. (by default, org id is your username)
my_activeloop_org_id = "your id"
my_activeloop_dataset_name = "langchain_youtube_summarizer"
dataset_path = f"hub://{my_activeloop_org_id}/{my_activeloop_dataset_name}"

db = DeepLake(dataset_path=dataset_path, embedding_function=embeddings)
db.add_documents(docs)

/home/gustavo-barbosa/anaconda3/envs/genai/lib/python3.11/site-packages/deeplake/util/check_latest_version.py:32: UserWarning: A newer version of deeplake (3.9.8) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(
/home/gustavo-barbosa/anaconda3/envs/genai/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(
Using embedding function is deprecated and will be removed in the future. Please use embedding instead.


Deep Lake Dataset in hub://gustavobarbosa060/langchain_youtube_summarizer already exists, loading from the storage


Creating 4 embeddings in 1 batches of size 4:: 100%|██████████| 1/1 [00:23<00:00, 23.13s/it]

Dataset(path='hub://gustavobarbosa060/langchain_youtube_summarizer', tensors=['embedding', 'id', 'metadata', 'text'])

  tensor      htype      shape     dtype  compression
  -------    -------    -------   -------  ------- 
 embedding  embedding  (8, 1536)  float32   None   
    id        text      (8, 1)      str     None   
 metadata     json      (8, 1)      str     None   
   text       text      (8, 1)      str     None   


['ed7ccd8c-1bac-11ef-b13e-8038fb44b484',
 'ed7ccdfa-1bac-11ef-b13e-8038fb44b484',
 'ed7cce22-1bac-11ef-b13e-8038fb44b484',
 'ed7cce36-1bac-11ef-b13e-8038fb44b484']

In [9]:
retriever = db.as_retriever()
retriever.search_kwargs['distance_metric'] = 'cos'
retriever.search_kwargs['k'] = 4

In [10]:
from langchain.prompts import PromptTemplate
prompt_template = """Use os seguintes trechos das transcrições de um vídeo para responder à pergunta em tópicos e resumido. Se você não souber a resposta, apenas diga que não sabe, não tente inventar uma resposta.

{context}

Pergunta: {question}
Resposta resumida em tópicos:"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [11]:
from langchain.chains import RetrievalQA

chain_type_kwargs = {"prompt": PROMPT}
qa = RetrievalQA.from_chain_type(llm=llm,
                                 chain_type="stuff",
                                 retriever=retriever,
                                 chain_type_kwargs=chain_type_kwargs)

print( qa.run("Resuma o por que NÃO sabemos QUASE NADA sobre o FUNDO do MAR?") )

/home/gustavo-barbosa/anaconda3/envs/genai/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(



- Não sabemos quase nada sobre o fundo do mar porque é um ambiente extremamente inacessível e hostil.
- A pressão nas profundezas do oceano é muito alta, o que dificulta a exploração.
- Além disso, a escuridão e a falta de oxigênio também são fatores que tornam difícil a exploração do fundo do mar.
- A tecnologia atual ainda não é avançada o suficiente para permitir uma exploração completa e detalhada do fundo do mar.
- A maior parte do fundo do mar ainda não foi mapeada ou estudada, o que contribui para o nosso desconhecimento sobre esse ambiente.
